In [2]:
import pandas as pd
from pandas.io import gbq

In [3]:
df = gbq.read_gbq("SELECT * FROM test_data.PRODUCT", project_id="opera-220120")

We have now connected to a database on Google BigQuery. Let's take a quick glance at the data we are going to work with

In [6]:
df.head()

,partition_date,timeperiod,country,full_version,news_locale,channel,source,impressions,clicks,active_users,new_users,retained_01,retained_07,retained_30
0,2018-06-02,2018-06-01,HF,3.3.2254.127986,hw:en,Cross-Promo,Unprecedented_2,NaN,NaN,NaN,NaN,4.0,NaN,NaN
1,2018-06-02,2018-06-01,BF,2.4.2254.127188,ba:fr,Cross-Promo,Widespread_4,NaN,NaN,NaN,NaN,3.0,NaN,NaN
2,2018-06-02,2018-06-01,BF,3.3.2254.127986,ba:fr,Cross-Promo,Widespread_4,NaN,NaN,NaN,NaN,10.0,NaN,NaN
3,2018-06-02,2018-06-01,JE,3.3.2254.127986,lv:fr,Cross-Promo,Widespread_4,NaN,NaN,NaN,NaN,11.0,NaN,NaN
4,2018-06-02,2018-06-01,JE,2.4.2254.127188,lv:fr,Cross-Promo,Widespread_4,NaN,NaN,NaN,NaN,4.0,NaN,NaN


The first thing we see is that there are several null values. Let us check how prevelent this is  

In [9]:
df["clicks"].isna().sum()/len(df["clicks"])

0.6235216056753986

So about 62% of the rows have null value for clicks. This seems realistic. Not all products in all coutries will have clicks

Let us find out the time range the data covers

In [13]:
max(df["timeperiod"])

Timestamp('2018-07-31 00:00:00')

In [14]:
min(df["timeperiod"])

Timestamp('2018-06-01 00:00:00')

The data covers two months. From first of June to 31st of July 2018.

In [19]:
out = df.groupby(["country", "partition_date"]).mean().groupby("country")["new_users"].mean()
out

country
BF     2.520582
DC     4.201086
GT     3.580905
HF     2.930920
IB     3.103561
JE     2.636894
JY     1.210033
KI     3.650642
LP     0.560923
LQ     1.829285
LY     0.360828
OU     0.884616
RE     4.531903
RO     2.030105
TT     1.774262
TZ     0.950392
UF     5.333772
WA     0.785179
XJ    10.649609
ZY    28.443589
Name: new_users, dtype: float64